Let's start by loading our data and doing a little feature engineering. Our data was stored with two header rows, and we don't need two header rows, so we need to change that. We need to convert our Date column to dates. And we want to extract the year; we will be using the years to indicate the passage of time in our scatter plots below.

In [1]:
import pandas as pd

AMZN = '/kaggle/input/amazon-stock-market-data-2015-2024/AMZN.csv'
df = pd.read_csv(filepath_or_buffer=AMZN, header=[0, 1])
df.columns = ['Date', 'Adj Close', 'Close', 'High', 'Lose', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df.head()

,Date,Adj Close,Close,High,Lose,Open,Volume,year
0,2015-01-02,15.4260,15.4260,15.7375,15.3480,15.6290,55664000,2015
1,2015-01-05,15.1095,15.1095,15.4190,15.0425,15.3505,55484000,2015
2,2015-01-06,14.7645,14.7645,15.1500,14.6190,15.1120,70380000,2015
3,2015-01-07,14.9210,14.9210,15.0640,14.7665,14.8750,52806000,2015
4,2015-01-08,15.0230,15.0230,15.1570,14.8055,15.0160,61768000,2015


Let's look at our correlations to see two things:
1. Do our adjusted close and close prices differ?
2. What do our price/volume correlations look like?

In [2]:
df[['Adj Close', 'Close', 'High', 'Lose', 'Open', 'Volume']].corr()

,Adj Close,Close,High,Lose,Open,Volume
Adj Close,1.000000,1.000000,0.999732,0.999740,0.999419,-0.194891
Close,1.000000,1.000000,0.999732,0.999740,0.999419,-0.194891
High,0.999732,0.999732,1.000000,0.999660,0.999763,-0.188121
Lose,0.999740,0.999740,0.999660,1.000000,0.999714,-0.201238
Open,0.999419,0.999419,0.999763,0.999714,1.000000,-0.193585
Volume,-0.194891,-0.194891,-0.188121,-0.201238,-0.193585,1.000000


Our correlations tell us two things:
* There are no adjustments in the adjusted close prices; this means we can use the close prices everywhere and ignore the adjusted close prices.
* Prices and volume are slightly negatively correlated, meaning that volume declines a little on average as prices rise.

Next let's look at our closing price time series.